In [1]:
import time
from ortools.sat.python import cp_model
import pandas as pd

# Definição das turmas e horários
turmas = ["1º A", "1º B", "1º C", "1º D", "2º A HUM", "2º B EXAT", "3º A HUM"]
dias = ["Seg", "Ter", "Qua", "Qui", "Sex"]
horarios = ["14:20 - 15:10", "15:10 - 16:00", "16:10 - 17:00", "17:00 - 17:50", "18:50 - 19:40", "19:40 - 20:30", "20:30 - 21:20"]

# Definindo a quantidade de aulas necessárias por turma
aulas_necessarias = {
    "1º A": {"Matematica": 4, "Portugues": 4, "Fisica": 2, "Biologia": 2, "Historia": 2, "Arte": 2, "EF": 2, "Geografia": 2, "Filosofia": 2, "OE": 2, "Quimica": 2, "Ed. Fin": 2, "Eletivas": 2, "Redação": 2, "PV": 1, "Ingles": 2},
    "1º B": {"Matematica": 4, "Portugues": 4, "Fisica": 2, "Biologia": 2, "Historia": 2, "Arte": 2, "EF": 2, "Geografia": 2, "Filosofia": 2, "OE": 2, "Quimica": 2, "Ed. Fin": 2, "Eletivas": 2, "Redação": 2, "PV": 1, "Ingles": 2},
    "1º C": {"Matematica": 4, "Portugues": 4, "Fisica": 2, "Biologia": 2, "Historia": 2, "Arte": 2, "EF": 2, "Geografia": 2, "Filosofia": 2, "OE": 2, "Quimica": 2, "Ed. Fin": 2, "Eletivas": 2, "Redação": 2, "PV": 1, "Ingles": 2},
    "1º D": {"Matematica": 4, "Portugues": 4, "Fisica": 2, "Biologia": 2, "Historia": 2, "Arte": 2, "EF": 2, "Geografia": 2, "Filosofia": 2, "OE": 2, "Quimica": 2, "Ed. Fin": 2, "Eletivas": 2, "Redação": 2, "PV": 1, "Ingles": 2},
    "2º A HUM": {"Matematica": 4, "Portugues": 4, "Fisica": 2, "Biologia": 2, "EF": 1, "PV": 1, "OE": 2, "Quimica": 2, "Ed. Fin": 2, "Eletivas": 2, "Sociologia": 2, "Liderança": 2, "Historia": 2, "Ingles": 2, "Oratoria": 1, "Redação": 2, "Geografia": 2},
    "2º B EXAT": {"Biologia": 2, "Ed. Fin": 2, "EF": 1, "Eletivas": 2, "Empreendedorismo": 1, "Fisica": 2, "Geografia": 2, "Historia": 2, "Ingles": 2, "Matematica": 4, "OE": 2, "Portugues": 4, "Programação": 2, "PV": 1, "Quimica": 2, "Redação": 2, "Sociologia": 2},
    "3º A HUM": {"AMD": 2, "EF": 1, "Eletivas": 2, "Filosofia": 2, "Fisica": 2, "Geopolitica": 2, "Historia": 2, "Ingles": 2, "Matematica": 4, "OE": 4, "Oratoria": 2, "PE": 3, "Portugues": 4, "PV": 1, "Redação": 2}
}

# Disciplinas por turma
disciplinas_por_turma = {
    "1º A": ["Matematica", "Portugues", "Fisica", "Biologia", "Historia", "Arte", "EF", "Geografia", "Filosofia", "OE", "Quimica", "Ed. Fin", "Eletivas", "Redação", "PV", "Ingles"],
    "1º B": ["Matematica", "Portugues", "Fisica", "Biologia", "Historia", "Arte", "EF", "Geografia", "Filosofia", "OE", "Quimica", "Ed. Fin", "Eletivas", "Redação", "PV", "Ingles"],
    "1º C": ["Matematica", "Portugues", "Fisica", "Biologia", "Historia", "Arte", "EF", "Geografia", "Filosofia", "OE", "Quimica", "Ed. Fin", "Eletivas", "Redação", "PV", "Ingles"],
    "1º D": ["Matematica", "Portugues", "Fisica", "Biologia", "Historia", "Arte", "EF", "Geografia", "Filosofia", "OE", "Quimica", "Ed. Fin", "Eletivas", "Redação", "PV", "Ingles"],
    "2º A HUM": ["Matematica", "Portugues", "Fisica", "Biologia", "EF", "PV", "OE", "Quimica", "Ed. Fin", "Eletivas", "Sociologia", "Liderança", "Historia", "Ingles", "Oratoria", "Redação", "Geografia"],
    "2º B EXAT": ["Biologia", "Ed. Fin", "EF", "Eletivas", "Empreendedorismo", "Fisica", "Geografia", "Historia", "Ingles", "Matematica", "OE", "Portugues", "Programação", "PV", "Quimica", "Redação", "Sociologia"],
    "3º A HUM": ["AMD", "EF", "Eletivas", "Filosofia", "Fisica", "Geopolitica", "Historia", "Ingles", "Matematica", "OE", "Oratoria", "PE", "Portugues", "PV", "Redação"]
}

# Professores e suas disciplinas
professores = {
    "LINCOLN": ["Matematica", "OE", "Eletiva"],
    "MONICA": ["Portugues", "Ingles", "Eletiva"],
    "LUCA": ["Matematica", "Fisica", "Eletiva", "Ed. Fin", "Programação", "Eletiva"],
    "FABIO": ["Matematica", "Filosofia", "OE", "Eletiva"],
    "MARIANA": ["Ed. Fin", "OE", "Eletiva"],
    "GUTO": ["Biologia", "Quimica", "Eletiva", "Empreendedorismo", "PE", "Eletiva"],
    "MARCIA": ["Historia", "Geografia", "Filosofia", "Geopolitica", "Eletiva"],
    "ROSANGELA": ["Arte", "AMD", "Eletiva"],
    "THAY": ["EF", "Eletiva", "PV", "Liderança", "Eletiva"],
    "ANA JULIA": ["Redação", "Oratoria", "OE", "Eletiva"],
    "BÁRBARA": ["OE", "Eletiva"],
    "HILARI": ["OE", "Redação", "Portugues", "Eletiva"],
    "CLARI": ["OE", "Eletiva"],
    "ADRIELI": ["Sociologia", "Eletivas"]
}

# Carga horária máxima por professor
aulas_maximas = {
    "LINCOLN": 30,
    "MONICA": 30,
    "LUCA": 30,
    "FABIO": 14,
    "MARIANA": 8,
    "GUTO": 30,
    "MARCIA": 30,
    "ROSANGELA": 12,
    "THAY": 26,
    "ANA JULIA": 20,
    "BÁRBARA": 4,
    "HILARI": 30,
    "CLARI": 4,
    "ADRIELI": 30
}


load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\zlib1.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\re2.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\highs.dll...
load C:\Users\lucad\anaconda3\Lib\site-packages\ortools\.libs\ortools.dll...


In [9]:
import time
import pandas as pd
from ortools.sat.python import cp_model

def resolver_alocacao(model, solver, turmas, dias, horarios, disciplinas_por_turma, professores, horario_disciplina):
    """Resolve o modelo de alocação e imprime a solução."""
    status = solver.Solve(model)
    
    if status == cp_model.OPTIMAL:
        print("Solução ótima encontrada!")
        for turma in turmas:
            for dia in dias:
                for horario in horarios:
                    for disciplina in disciplinas_por_turma.get(turma, []):
                        for professor in professores:
                            if solver.BooleanValue(horario_disciplina.get((turma, dia, horario, disciplina, professor))):
                                print(f"{turma} - {disciplina} - {professor} em {dia} {horario}")
    else:
        print("Não foi possível encontrar uma solução ótima.")


def criar_modelo_e_solver():
    """Cria o modelo e o solver."""
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    return model, solver

def criar_variaveis_decisao(model, turmas, dias, horarios, disciplinas_por_turma, professores):
    """Cria as variáveis de decisão para alocação de horários."""
    horario_disciplina = {}
    for turma in turmas:
        for dia in dias:
            for horario in horarios:
                for disciplina in disciplinas_por_turma.get(turma, []):
                    for professor in professores:
                        if disciplina in professores[professor]:
                            var = model.NewBoolVar(f"{turma}_{dia}_{horario}_{disciplina}_{professor}")
                            horario_disciplina[(turma, dia, horario, disciplina, professor)] = var
    return horario_disciplina

def adicionar_restricoes_turma_disciplina(model, horario_disciplina, turmas, dias, horarios, disciplinas_por_turma, professores):
    """Adiciona as restrições de que cada turma deve ter uma disciplina por horário."""
    for turma in turmas:
        for dia in dias:
            for horario in horarios:
                model.Add(
                    sum(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)
                        for disciplina in disciplinas_por_turma.get(turma, [])
                        for professor in professores) == 1
                )

def adicionar_restricoes_carga_horaria(model, horario_disciplina, turmas, dias, horarios, professores, aulas_maximas):
    """Adiciona as restrições de carga horária dos professores."""
    for professor in professores:
        model.Add(
            sum(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)
                for turma in turmas
                for dia in dias
                for horario in horarios
                for disciplina in professores[professor]) <= aulas_maximas[professor]
        )

def adicionar_restricoes_professor_exclusivo(model, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma):
    """Garante que cada disciplina tenha apenas um professor atribuído."""
    for turma in turmas:
        for disciplina in disciplinas_por_turma[turma]:
            professores_habilitados = [prof for prof in professores if disciplina in professores[prof]]
            if len(professores_habilitados) > 1:
                professor_selecionado = model.NewIntVar(0, len(professores_habilitados) - 1, f'prof_selecionado_{turma}_{disciplina}')
                
                for i, professor in enumerate(professores_habilitados):
                    professor_alocado = model.NewBoolVar(f'prof_alocado_{professor}_{turma}_{disciplina}')
                    
                    model.AddBoolOr([
                        horario_disciplina.get((turma, dia, horario, disciplina, professor), None)
                        for dia in dias
                        for horario in horarios
                    ]).OnlyEnforceIf(professor_alocado)
                    
                    model.Add(
                        sum(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)
                            for dia in dias
                            for horario in horarios) == 0
                    ).OnlyEnforceIf(professor_alocado.Not())
                    
                    model.Add(professor_selecionado == i).OnlyEnforceIf(professor_alocado)
                    model.Add(professor_selecionado != i).OnlyEnforceIf(professor_alocado.Not())

def adicionar_restricoes_aulas_necessarias(model, horario_disciplina, turmas, dias, horarios, aulas_necessarias, professores):
    """Adiciona a restrição de número de aulas necessárias por turma e disciplina."""
    for turma in turmas:
        for disciplina, qtd_aulas in aulas_necessarias.get(turma, {}).items():
            model.Add(
                sum(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)
                    for dia in dias
                    for horario in horarios
                    for professor in professores if disciplina in professores[professor]) == qtd_aulas
            )

def executar_solver(model, solver):
    """Executa o solver e retorna o tempo de execução e o status da solução."""
    start_time = time.time()
    status = solver.Solve(model)
    execution_time = time.time() - start_time
    return status, execution_time

def gerar_horarios_por_turma(solver, horario_disciplina, turmas, dias, horarios, disciplinas_por_turma, professores):
    """Gera os horários por turma formatados para exibição."""
    horarios_por_turma = {turma: {horario: {dia: '' for dia in dias} for horario in horarios} for turma in turmas}
    for turma in turmas:
        for dia in dias:
            for horario in horarios:
                for disciplina in disciplinas_por_turma.get(turma, []):
                    for professor in professores:
                        if solver.Value(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)) == 1:
                            horarios_por_turma[turma][horario][dia] = f"{disciplina} - {professor}"
    return horarios_por_turma


def salvar_horarios_em_excel(turmas, horarios_por_turma):
    """Salva os horários em um arquivo Excel, com abas para cada turma."""
    with pd.ExcelWriter('horarios_por_turma.xlsx') as writer:
        for turma in turmas:
            df = pd.DataFrame(horarios_por_turma[turma]).T
            df.index.name = "Horário"
            df.columns.name = "Dia"
            df.fillna('', inplace=True)
            df.to_excel(writer, sheet_name=f'Horários{turma}')


def exibir_horarios(turmas, horarios_por_turma):
    """Exibe os horários em formato de tabela para cada turma e salva como Excel."""
    for turma in turmas:
        print(f"\nHorário da turma {turma}:")
        df = pd.DataFrame(horarios_por_turma[turma]).T
        df.index.name = "Horário"
        df.columns.name = "Dia"
        display(df.fillna(''))
        #df.to_excel(f"horarios_turma_{turma}.xlsx")

def identificar_conflitos(solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma):
    """Identifica conflitos de horário nas alocações."""
    conflitos = []
    for turma in turmas:
        for dia in dias:
            for horario in horarios:
                professores_atribuidos = []
                disciplinas_atribuidas = []
                for disciplina in disciplinas_por_turma.get(turma, []):
                    for professor in professores:
                        if solver.Value(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)) == 1:
                            professores_atribuidos.append(professor)
                            disciplinas_atribuidas.append(disciplina)
                if len(professores_atribuidos) > 1:
                    conflitos.append((turma, dia, horario, list(zip(professores_atribuidos, disciplinas_atribuidas))))
    return conflitos

def exibir_conflitos(conflitos):
    """Exibe os conflitos de horário."""
    if conflitos:
        print("\nConflitos de Horário:")
        for conflito in conflitos:
            print(f"Turma: {conflito[0]}, Dia: {conflito[1]}, Horário: {conflito[2]}, Professores e Disciplinas: {conflito[3]}")
    else:
        print("\nNão há conflitos de horário.")

def identificar_problemas_professores(solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma):
    """Identifica professores com mais problemas de horário."""
    professores_problemas = {}
    for professor in professores:
        problemas = 0
        for turma in turmas:
            for dia in dias:
                for horario in horarios:
                    for disciplina in professores[professor]:
                        if solver.Value(horario_disciplina.get((turma, dia, horario, disciplina, professor), 0)) == 1:
                            problemas += 1
        professores_problemas[professor] = problemas
    return professores_problemas

def exibir_problemas_professores(professores_problemas):
    """Exibe os professores com mais problemas de horário."""
    print("\nProfessores com mais problemas de horário:")
    for professor, problemas in professores_problemas.items():
        if problemas > 0:
            print(f"{professor}: {problemas} problemas")

def verificar_solucao(status, solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma, aulas_necessarias):
    """Verifica a solução e exibe os resultados."""
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print("Solução encontrada!")
        
        horarios_por_turma = gerar_horarios_por_turma(solver, horario_disciplina, turmas, dias, horarios, disciplinas_por_turma, professores)
        exibir_horarios(turmas, horarios_por_turma)
        salvar_horarios_em_excel(turmas, horarios_por_turma)
        
        conflitos = identificar_conflitos(solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma)
        exibir_conflitos(conflitos)
        
        professores_problemas = identificar_problemas_professores(solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma)
        exibir_problemas_professores(professores_problemas)
    else:
        print("Não foi possível encontrar uma solução viável.")

def main():
    # Criando modelo e solver
    model, solver = criar_modelo_e_solver()

    # Criando variáveis de decisão
    horario_disciplina = criar_variaveis_decisao(model, turmas, dias, horarios, disciplinas_por_turma, professores)

    # Adicionando restrições
    adicionar_restricoes_turma_disciplina(model, horario_disciplina, turmas, dias, horarios, disciplinas_por_turma, professores)
    adicionar_restricoes_carga_horaria(model, horario_disciplina, turmas, dias, horarios, professores, aulas_maximas)
    adicionar_restricoes_professor_exclusivo(model, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma)
    adicionar_restricoes_aulas_necessarias(model, horario_disciplina, turmas, dias, horarios, aulas_necessarias, professores)

    # Resolver modelo e exibir resultados
    status, execution_time = executar_solver(model, solver)
    print(f"\nTempo de execução: {execution_time:.2f} segundos")
    verificar_solucao(status, solver, horario_disciplina, turmas, dias, horarios, professores, disciplinas_por_turma, aulas_necessarias)

if __name__ == '__main__':
    main()



Tempo de execução: 1.23 segundos
Solução encontrada!

Horário da turma 1º A:


Dia,Seg,Ter,Qua,Qui,Sex
Horário,,,,,
14:20 - 15:10,Ingles - MONICA,Filosofia - FABIO,Redação - HILARI,EF - THAY,Matematica - FABIO
15:10 - 16:00,Fisica - LUCA,Quimica - GUTO,OE - ANA JULIA,Ed. Fin - MARIANA,Portugues - MONICA
16:10 - 17:00,Historia - MARCIA,Geografia - MARCIA,Redação - HILARI,Matematica - FABIO,Ingles - MONICA
17:00 - 17:50,Portugues - MONICA,Matematica - FABIO,Eletivas - ADRIELI,OE - ANA JULIA,Portugues - MONICA
18:50 - 19:40,Eletivas - ADRIELI,Fisica - LUCA,EF - THAY,Arte - ROSANGELA,Quimica - GUTO
19:40 - 20:30,Ed. Fin - MARIANA,Biologia - GUTO,PV - THAY,Filosofia - FABIO,Historia - MARCIA
20:30 - 21:20,Arte - ROSANGELA,Portugues - MONICA,Geografia - MARCIA,Biologia - GUTO,Matematica - FABIO



Horário da turma 1º B:


Dia,Seg,Ter,Qua,Qui,Sex
Horário,,,,,
14:20 - 15:10,EF - THAY,Geografia - MARCIA,Redação - HILARI,Fisica - LUCA,Geografia - MARCIA
15:10 - 16:00,Biologia - GUTO,Matematica - LINCOLN,OE - FABIO,PV - THAY,Filosofia - FABIO
16:10 - 17:00,Eletivas - ADRIELI,Quimica - GUTO,Ingles - MONICA,Portugues - MONICA,Historia - MARCIA
17:00 - 17:50,Eletivas - ADRIELI,Ed. Fin - LUCA,Matematica - LINCOLN,Quimica - GUTO,EF - THAY
18:50 - 19:40,Fisica - LUCA,Biologia - GUTO,Matematica - LINCOLN,Historia - MARCIA,Matematica - LINCOLN
19:40 - 20:30,OE - FABIO,Ed. Fin - LUCA,Arte - ROSANGELA,Portugues - MONICA,Redação - HILARI
20:30 - 21:20,Ingles - MONICA,Filosofia - FABIO,Portugues - MONICA,Arte - ROSANGELA,Portugues - MONICA



Horário da turma 1º C:


...


Horário da turma 2º B EXAT:


Dia,Seg,Ter,Qua,Qui,Sex
Horário,,,,,
14:20 - 15:10,Empreendedorismo - GUTO,Portugues - HILARI,Biologia - GUTO,Sociologia - ADRIELI,Programação - LUCA
15:10 - 16:00,Eletivas - ADRIELI,Portugues - HILARI,Fisica - LUCA,Fisica - LUCA,Ed. Fin - LUCA
16:10 - 17:00,Portugues - HILARI,Ingles - MONICA,Geografia - MARCIA,Geografia - MARCIA,Portugues - HILARI
17:00 - 17:50,Historia - MARCIA,Quimica - GUTO,Sociologia - ADRIELI,Redação - ANA JULIA,Programação - LUCA
18:50 - 19:40,Matematica - LINCOLN,Biologia - GUTO,Matematica - LINCOLN,Ed. Fin - LUCA,EF - THAY
19:40 - 20:30,Eletivas - ADRIELI,Matematica - LINCOLN,Matematica - LINCOLN,Ingles - MONICA,PV - THAY
20:30 - 21:20,Quimica - GUTO,Historia - MARCIA,Redação - ANA JULIA,OE - LINCOLN,OE - LINCOLN



Horário da turma 3º A HUM:


Dia,Seg,Ter,Qua,Qui,Sex
Horário,,,,,
14:20 - 15:10,Geopolitica - MARCIA,Oratoria - ANA JULIA,OE - ANA JULIA,Matematica - LINCOLN,Historia - MARCIA
15:10 - 16:00,Eletivas - ADRIELI,OE - ANA JULIA,Filosofia - MARCIA,PE - GUTO,Filosofia - MARCIA
16:10 - 17:00,AMD - ROSANGELA,Ingles - MONICA,Oratoria - ANA JULIA,Eletivas - ADRIELI,Historia - MARCIA
17:00 - 17:50,Redação - HILARI,Portugues - HILARI,Redação - HILARI,OE - ANA JULIA,Fisica - LUCA
18:50 - 19:40,Matematica - LINCOLN,PE - GUTO,OE - ANA JULIA,Portugues - HILARI,Portugues - HILARI
19:40 - 20:30,Fisica - LUCA,Matematica - LINCOLN,PV - THAY,Ingles - MONICA,EF - THAY
20:30 - 21:20,AMD - ROSANGELA,PE - GUTO,Matematica - LINCOLN,Geopolitica - MARCIA,Portugues - HILARI



Não há conflitos de horário.

Professores com mais problemas de horário:
LINCOLN: 26 problemas
MONICA: 30 problemas
LUCA: 26 problemas
FABIO: 14 problemas
MARIANA: 2 problemas
GUTO: 28 problemas
MARCIA: 30 problemas
ROSANGELA: 10 problemas
THAY: 20 problemas
ANA JULIA: 15 problemas
BÁRBARA: 2 problemas
HILARI: 22 problemas
CLARI: 2 problemas
ADRIELI: 18 problemas
